# Soynlp
soynlp는 한국어 처리를 위한 파이썬 패키지 

koNLPy에서 제공하는 형태소분석기는 형태소 기반으로 문서를 토큰화할 수 있는 기능을 제공하지만 새롭게 만들어진 미등록 단어들은 인식이 잘 되지 않는 단점을 가지고 있다.

이를 해결하기 위해서는 사용자 사전에 단어를 등록하는 절차를 거쳐야 한다. 

soynlp는 이러한 과정을 돕기 위해 사용자 사전과 형태소분석 없이 cohesion 기반으로 토큰화를 할 수 있는 기능을 제공한다. 

soynlp 에서 제공하는 WordExtractor /  NounExtractor 는 여러 개의 문서로부터 학습한 통계 정보를 이용하여 작동하기 때문에 하나의 문장 혹은 문서에서 보다는 어느 정도 규모가 있는 동일한 집단의 문서 (homogeneous documents) 에서 잘 작동한다.

- pip install soynlp

# 사전 미등록 단어의 문제

In [30]:
import sys
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum

kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
print('konlpy version = %s' % konlpy.__version__)

konlpy version = 0.5.2


In [31]:
sent = '박근혜 최순실의 국정농단 사태는 대통령탄핵으로 이어졌습니다'
print('꼬꼬마 명사: ', kkma.nouns(sent)) #고유명사 명사혼동함
print('OKT   명사: ', okt.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['박근', '박근혜', '혜', '최', '최순실의', '순', '실의', '국정', '국정농단', '농단', '사태', '대통령', '대통령탄핵', '탄핵']
OKT   명사:  ['박근혜', '최', '실의', '국정', '농단', '사태', '대통령', '탄핵']
한나눔 명사:  ['박근혜', '최순실', '국정농단', '사태', '대통령탄핵', '이']


'박근혜'의 경우 여러 정치 관련 문서에서 등장하였으나, '최순실'의 경우 학습데이터에 이 데이터가 들어가지 못해 대부분의 형태소 분석기가 해당 명사를 인식하지 못한다. 단, 꼬꼬마의 경우 고유명사를 잘 인식하지 못함

In [32]:
sent = '트와이스 그리고 아이오아이 좋아여 tt가 저번에 1위 했었죠?'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('OKT   명사: ', okt.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['트와', '트와이스', '이스', '아이오', '아이오아이', '아이', '여', '저번', '1', '1위', '위']
OKT   명사:  ['트와이스', '아이오', '아이', '저번', '위']
한나눔 명사:  ['트와이스', '아이오아', '좋아', '저번', '1위']


'트와이스', '아이오아이'처럼 시간이 지나면서 발생하는 단어들은 명사로 잘 인식되지 않음.
TT의 경우 외국어이기때문에 명사로 인식되지 못함.

In [33]:
sent = '보코하람 테러로 소말리아에서 전쟁이 있었어요'
print('꼬꼬마 명사: ', kkma.nouns(sent))
print('OKT   명사: ', okt.nouns(sent))
print('한나눔 명사: ', hannanum.nouns(sent))

꼬꼬마 명사:  ['보', '보코', '코', '테러', '소말리', '전쟁']
OKT   명사:  ['보코하람', '테러', '소말리아', '전쟁']
한나눔 명사:  ['보코하람', '테러', '소말리아', '전쟁']


'보코하람'은 단어로 알지 못하지만 '보', '코' 라는 것은 명사로 알고 있기 때문에 외국어를 분해

# L-R 구조를 이용한 명사 추출

- 오늘 A에서 저녁먹자

- 너랑간 A 진짜 좋더라

- 기다릴테니까 곧바로 A로 와

- 난 A가 좋더라

### 한국어는 L + [R] 구조
L 옆에 등장하는 R의 분포는 L이 명사인지 아닌지를 판단하는 좋은 힌트.
하지만 조사들을 R로 가지고 있으면서 규칙기반으로 명사를 찾을 수는 없다.



##  L - R의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단

세종 말뭉치를 이용하여 명사 뒤에 등장하는 R set을 모아뒀으며, R set의 단어들은 명사 가능 점수가 학습되어 있다. 명사 가능 점수의 범위는 [-1, 1]

- 내서     -0.530702

- 있게      1.000000

- 있는      0.327824

- 쓰는      0.079298

- 었다며    -1.000000

- 였다며     0.437399

    - '했 + 었다며'이라면 '했'은 명사 점수가 -1.0입니다. 
    - '재미 + 있게' 3번, '재미 + 있는' 2번 등장하였다면 재미의 명사 가능 점수는 (3 x 1.0 + 2 x 0.33) / 5 = 0.732점입니다. 명사 추출의 threshold가 0.5라면 '재미'는 명사로 추출됩니다.

이 방법은 주어진 문서집합에서 어절들의 구조를 학습하여 그 주어진 문서집합의 명사를 추출합니다. 학습데이터가 필요하지 않은 통계 기반의 unsupervised 학습방법입니다.


## LRNounExtractor

In [34]:
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded


In [38]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = '2016-10-20.txt' #  2016년 10월 20일의 뉴스
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

223357

추출하고 싶은 명사의 noun score threshold와 명사의 최소빈도수 (min count)를 parameter로 넣으면 명사 점수 0.3점 이상, 문서집합에서 등장한 빈도가 100번 이상인 단어들에서 명사를 추출

train 함수에서는 WordExtractor를 통하여 단어를 미리 추출하고, 이들 중에서 명사를 판단합니다

In [39]:
%%time
nouns = noun_extractor.train_extract(
    sentences,
    min_noun_score=0.3,
    min_noun_frequency=20
)

[Noun Extractor] scanning was done (L,R) has (52264, 26090) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 14589 nouns are extracted
Wall time: 55.7 s


In [40]:
# NounScore (frequency= 어절의 왼쪽에 등장한 횟수, score=명사 점수, NounScore_v1(frequency=8325, score=0.43977009340659345, known_r_ratio = R set 이 알려진 feature 인 비율)
nouns['뉴스']

NounScore_v1(frequency=8325, score=0.43977009340659345, known_r_ratio=0.052089295935890095)

In [41]:
#해당 단어가 명사로 추출되는지 확인
words = ['박근혜', '우병우', '민정수석', '트와이스', '아이오아이']
for word in words:
    print('%s is noun? %r' % (word, word in nouns))

박근혜 is noun? True
우병우 is noun? True
민정수석 is noun? True
트와이스 is noun? False
아이오아이 is noun? True


In [43]:
# 특정 단어 명사여부 확인
noun_extractor.is_noun('최순실', min_noun_score=0.3) 

True

In [44]:
# 명사점수 추출

words = ['박근혜', '우병우', '민정수석', 
         '트와이스', '아이오아이', '최순실',
         '최순실게이트', '게이트', '콘서트']

for word in words:
    noun_score, known_r_ratio = noun_extractor.predict(word)
    print('%s : score=%.3f' % (word, noun_score))

박근혜 : score=0.538
우병우 : score=0.757
민정수석 : score=0.834
트와이스 : score=0.262
아이오아이 : score=0.980
최순실 : score=0.828
최순실게이트 : score=0.320
게이트 : score=0.745
콘서트 : score=0.769


In [46]:
# 빈도수 기준 상위 50개
top100 = sorted(nouns.items(), 
    key=lambda x:-x[1].frequency)[:50]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.score), end='')


    기자 (0.67)    무단 (1.00)   20일 (0.91)   재배포 (1.00)    금지 (0.99)
    사진 (0.40)    지난 (0.99)   뉴시스 (1.00)    뉴스 (0.44)    함께 (0.97)
    대표 (0.39)    진행 (0.92)    오후 (0.99)  저작권자 (1.00)    때문 (1.00)
    이번 (0.99)    미국 (0.77)    이날 (0.69)    관련 (0.96)    참석 (0.98)
    제공 (0.96)   대통령 (0.49)   19일 (0.88)   이라고 (0.99)    방송 (0.80)
    오전 (1.00)    우리 (0.53)    경찰 (0.32)    영상 (0.54)    사람 (0.52)
    공감 (0.69)    최근 (0.98)  연합뉴스 (0.69)    모습 (0.73)    공개 (0.98)
    하고 (0.80)    사랑 (0.91)    시작 (0.96)    같은 (1.00)    올해 (0.39)
    가장 (0.97)    이후 (0.98)    한국 (0.78)    경우 (0.66)    라고 (1.00)
    문제 (0.68)    이상 (0.86)    정부 (0.57)    제보 (1.00)    하지 (1.00)

In [47]:
# 빈도수* 명사점수 기준 상위 50개
top100 = sorted(nouns.items(), 
    key=lambda x:-x[1].frequency * x[1].score)[:50]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.score), end='')


    무단 (1.00)   재배포 (1.00)    금지 (0.99)   20일 (0.91)    기자 (0.67)
    지난 (0.99)   뉴시스 (1.00)    함께 (0.97)    오후 (0.99)  저작권자 (1.00)
    진행 (0.92)    때문 (1.00)    이번 (0.99)    관련 (0.96)    참석 (0.98)
   이라고 (0.99)    제공 (0.96)    오전 (1.00)    사진 (0.40)    미국 (0.77)
   19일 (0.88)    최근 (0.98)    공개 (0.98)    같은 (1.00)    이날 (0.69)
    방송 (0.80)    라고 (1.00)    시작 (0.96)    이후 (0.98)    가장 (0.97)
    제보 (1.00)    하지 (1.00)    사랑 (0.91)    다양 (1.00)    국내 (0.97)
    자신 (0.90)    뉴스 (0.44)    이상 (0.86)    세계 (0.98)    하고 (0.80)
   있다고 (1.00)    한국 (0.78)    모습 (0.73)    한편 (0.98)    북한 (0.80)
    공감 (0.69)    국회 (0.95)    확인 (0.95)  연합뉴스 (0.69)    당시 (0.99)